# Dataset Preprocessing

In [ ]:
import pandas as pd
import os

## Loading both datasets

### Depmap cancer cell lines omics data

In [ ]:
# Metadata file
meta_df = pd.read_csv(os.path.join("data", "Model.csv"), header=0)

In [ ]:
meta_df.info()

In [ ]:
meta_df.loc[:10, ["ModelID", "StrippedCellLineName"]]

In [ ]:
# Omics datafile for cancer cell lines
genes_expressions_path = os.path.join(
    "data", "OmicsExpressionProteinCodingGenesTPMLogp1.csv"
)
ge_df = pd.read_csv(genes_expressions_path, header=0)
ge_df.rename(columns={"Unnamed: 0": "ModelID"}, inplace=True)
ge_df.loc[:10, ["ModelID"]]

In [ ]:
ge_names = list(ge_df.columns)
ge_names.remove("ModelID")
ge_names[:10]

In [ ]:
# Merge metadata and omics data
ge_df = pd.merge(meta_df, ge_df, how="inner", on="ModelID")
ge_df.loc[:10, ["ModelID", "StrippedCellLineName"]]

In [ ]:
ge_df.head()

In [ ]:
ge_df.drop(
    [k for k in ge_df.columns if k not in [*ge_names, "StrippedCellLineName"]],
    axis=1,
    inplace=True,
)
ge_df.rename(columns={"StrippedCellLineName": "ccl_name"}, inplace=True)
ge_df.head()

In [ ]:
ge_df.to_csv(os.path.join("data", "ge.csv"), index=False)

### CTRPv2 DRP experiment

In [ ]:
drp_data = pd.read_csv(
    os.path.join("data", "CTRPv2", "v21.data.auc_sensitivities.txt"), sep="\t", header=0
)
drp_data.head()

In [ ]:
meta_cell_lines = pd.read_csv(
    os.path.join("data", "CTRPv2", "v21.meta.per_cell_line.txt"),
    sep="\t",
    header=0,
    usecols=["ccl_name", "master_ccl_id"],
)
meta_cell_lines.head()

In [ ]:
len(meta_cell_lines)

In [ ]:
meta_compounds = pd.read_csv(
    os.path.join("data", "CTRPv2", "v21.meta.per_compound.txt"),
    sep="\t",
    header=0,
    usecols=["cpd_smiles", "master_cpd_id"],
)
meta_compounds.head()

In [ ]:
len(meta_compounds)

In [ ]:
# merge drp data with meta compounds and meta cell lines
drp_data = drp_data.merge(
    meta_cell_lines, left_on="master_ccl_id", right_on="master_ccl_id", how="left"
)
drp_data = drp_data.merge(
    meta_compounds, left_on="master_cpd_id", right_on="master_cpd_id", how="left"
)
drp_data.drop(["master_ccl_id", "master_cpd_id", "experiment_id"], axis=1, inplace=True)
drp_data.head()

In [ ]:
len(drp_data)

In [ ]:
drp_data.to_csv(os.path.join("data", "drp.csv"), index=False)

### Removing diff in ccl_names between DRP and GE datasets

When exploring both the DRP and GE datasets, we noted that certain cell lines were appearing in the DRP dataset but not in the GE dataset.

As they represent a very marginal fraction of the total number of cell lines tested, we decided to drop them from the DRP dataset.

In [ ]:
ge_df = pd.read_csv(os.path.join("data", "ge.csv"), header=0, index_col=0)
drp_df = pd.read_csv(os.path.join("data", "drp.csv"), header=0)

In [ ]:
drp_df.shape

In [ ]:
ccl_names_drp = list(drp_df['ccl_name'].unique())
ccl_names_ge = list(ge_df.index.unique())
ccl_names_diff = list(set(ccl_names_drp) - set(ccl_names_ge))
print(len(ccl_names_diff))

In [ ]:
drp_df.drop(drp_df[drp_df['ccl_name'].isin(ccl_names_diff)].index, inplace=True)

In [ ]:
drp_df.shape

In [ ]:
drp_df.to_csv(os.path.join("data", "drp.csv"), index=False)

## SMILES to SELFIES

In [ ]:
import selfies as sf
import joblib
from joblib import Parallel, delayed
from multiprocessing import cpu_count

Generating selfies for train set :

In [ ]:
drp_df = pd.read_csv(os.path.join("data", "drp.csv"), header=0)
drp_df.head()

In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

def apply_func_to_chunk(chunk, func):
    try:
        chunk['cpd_selfies'] = chunk['cpd_smiles'].apply(func)
        return chunk
    except Exception as e:
        print(f"Error: {e}")
        return chunk

def parallel_apply(df, func, column):
    num_cores = joblib.cpu_count()
    df_split = np.array_split(df, num_cores)

    results = Parallel(n_jobs=num_cores)(delayed(apply_func_to_chunk)(chunk, func) for chunk in df_split)

    df = pd.concat(results)
    return df

In [ ]:
drp_df = parallel_apply(drp_df, sf.encoder, 'cpd_smiles') # parallel apply if possible
# drp_df['cpd_selfies'] = drp_df['cpd_smiles'].apply(sf.encoder) # if not possible


In [ ]:
print(len(drp_df))
drp_df.head()

In [ ]:
drp_df.rename(columns={"cpd_smiles": "smiles", "cpd_selfies" : "selfies"}, inplace=True)
drp_df.head()

In [ ]:
drp_df.to_csv(os.path.join("data", "drp.csv"), index=False)

## SMILES to Morgan Fingerprints

In [ ]:
import os
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import joblib

In [ ]:
drp_df = pd.read_csv(os.path.join("data", "drp.csv"), header=0)

In [ ]:
smiles_unique = list(np.unique(drp_df["smiles"].values))
print(f"Number of unique smiles: {len(smiles_unique)}")

In [ ]:
smiles_unique[0]

In [ ]:
bit={}
smiles_fingerprints = []
for smiles in smiles_unique:
    m = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,useChirality=True, radius=3, nBits = 2048, bitInfo=bit)
    smiles_fingerprints.append(np.array(fp))

In [ ]:
len(smiles_fingerprints), smiles_fingerprints[0].shape

In [ ]:
compound_embeddings_dict = {s: embedding for s, embedding in zip(smiles_unique, smiles_fingerprints)}
compound_embeddings_dict

In [ ]:
joblib.dump(compound_embeddings_dict, os.path.join("data", "smiles_fingerprints_embeddings_dict.joblib"))

In [ ]:
def test_embeddings_presence(drp_path, embeddings_dict_path):
    drp_df = pd.read_csv(drp_path, header=0)
    embeddings_dict = joblib.load(embeddings_dict_path)

    smiles_unique = set(drp_df["smiles"].values)
    smiles_embeddings = set(embeddings_dict.keys())
    print(f"Number of unique smiles: {len(smiles_unique)}")
    print(f"Number of smiles in embeddings_dict: {len(smiles_embeddings)}")
    
    return True if smiles_unique.issubset(smiles_embeddings) else False

In [ ]:
test_embeddings_presence(os.path.join("data", "drp_train.csv"), os.path.join("data", "smiles_fingerprints_embeddings_dict.joblib"))

## Splitting DRP data in train and test sets

We want to take a drug-blind evaluation approach, which entails keeping a number of drugs unseen to the model during training. 

In [ ]:
# fix the seed
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(0)

In [ ]:
drp_df = pd.read_csv(os.path.join("data", "drp.csv"), header=0)
smiles = drp_df["smiles"].unique()

In [ ]:
print(len(smiles))

In [ ]:
train_size = int(0.8 * len(smiles))
test_size = len(smiles) - train_size
print(f"Number of training compounds : {train_size}")
print(f"Number of test compounds : {test_size}")

In [ ]:
train_compounds, test_compounds = train_test_split(
    smiles,
    train_size=train_size,
    test_size=test_size,
    random_state=0,
)

In [ ]:
drp_train = drp_df[drp_df['smiles'].isin(train_compounds)]
drp_test = drp_df[drp_df['smiles'].isin(test_compounds)]

In [ ]:
len(drp_train), len(drp_test)

In [ ]:
# save all dfs to disk
drp_train.to_csv(os.path.join("data", "drp_train.csv"), index=False)
drp_test.to_csv(os.path.join("data", "drp_test.csv"), index=False)

## GE filtering

Gene expressions taken from Depmap dataset are already partially preprocessed.

Here we will only remove the gene expressions with low variance (<1), which we will hypothetize to have low importance in the cancer biological processes.

The variance will be computed on the train set only to prevent any data leakage to the test set. Then the selection obtained from the train set will be applied to the test set.

In [ ]:
import joblib
from joblib import Parallel, delayed
from multiprocessing import cpu_count
from typing import List

In [ ]:
def process_chunk(chunk, threshold):
    variances = chunk.var(axis=0)
    return variances.index[variances >= threshold].tolist()

def filter_low_variance_genes_large_dataset(filepath, threshold: float = 1.0) -> List[str]:
    chunk_size = 10 ** 4
    chunks = pd.read_csv(filepath, chunksize=chunk_size, index_col=0, header=0)
    
    results = Parallel(n_jobs=cpu_count())(delayed(process_chunk)(chunk, threshold) for chunk in chunks)
    
    columns_to_keep = list(set().union(*results))
    return columns_to_keep

In [ ]:
ge_path = os.path.join("data", "ge.csv")
columns_to_keep = filter_low_variance_genes_large_dataset(ge_path, threshold=1.0)
print(len(columns_to_keep))

In [ ]:
ge_filtered = pd.read_csv(ge_path, usecols=['ccl_name', *columns_to_keep], header=0, index_col=0)
ge_filtered.head()

In [ ]:
ge_filtered.shape

In [ ]:
ge_filtered.to_csv(os.path.join("data", "ge_filtered.csv"), index=True)

In [ ]:
joblib.dump(columns_to_keep, os.path.join("data", "list_genes_filtered.joblib"))

## Features Scaling

Let's normalize both the AUC label score and the GE features of the train set.

### GE Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib

In [ ]:
ge_filtered_df = pd.read_csv(os.path.join("data", "ge_filtered.csv"), header=0, index_col=0)
ge_filtered_df.head()

In [ ]:
features = ge_filtered_df.values
features.shape

In [ ]:
# scaler = StandardScaler() # Normal distribution, preserves outliers that may be way past [-1, 1]
scaler = MinMaxScaler((-1, 1)) # fit between -1 and 1, preserves outliers
normalized_data = scaler.fit_transform(features)

In [ ]:
# save dataset with scaled features
ge_filtered_scaled_df = ge_filtered_df.copy()
ge_filtered_scaled_df.iloc[:, :] = normalized_data
ge_filtered_scaled_df.head()

In [ ]:
# save dataset with scaled features
ge_filtered_scaled_df.to_csv(os.path.join("data", "ge_filtered_scaled.csv"), index=True)

In [ ]:
# save the scaler checkpoint
joblib.dump(scaler, os.path.join("data", "scaler_checkpoints", "ge_scaler.joblib"))

### AUC Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib

In [ ]:
drp_train_df = pd.read_csv(os.path.join("data", "drp_train.csv"), header=0)
drp_train_df.head()

In [ ]:
auc_values = drp_train_df["area_under_curve"].values
auc_values.shape

In [ ]:
# scaler = StandardScaler() # Normal distribution, preserves outliers that may be way past [-1, 1]
scaler = MinMaxScaler((-1, 1)) # fit between -1 and 1, preserves outliers
normalized_data = scaler.fit_transform(auc_values.reshape(-1, 1)).squeeze()
normalized_data.shape

In [ ]:
drp_train_df["area_under_curve_scaled"] = normalized_data
drp_train_df = drp_train_df[["area_under_curve", "area_under_curve_scaled", "ccl_name", "smiles", "selfies"]]
drp_train_df.head()

In [ ]:
# save dataset with scaled features
drp_train_df.to_csv(os.path.join("data", "drp_train.csv"), index=False)

In [ ]:
# apply the transformation to the test set
drp_test_df = pd.read_csv(os.path.join("data", "drp_test.csv"), header=0)
drp_test_df.head()

In [ ]:
auc_values = drp_test_df["area_under_curve"].values
auc_values.shape

In [ ]:
normalized_data = scaler.transform(auc_values.reshape(-1, 1)).squeeze()
normalized_data.shape

In [ ]:
drp_test_df["area_under_curve_scaled"] = normalized_data
drp_test_df = drp_test_df[["area_under_curve", "area_under_curve_scaled", "ccl_name", "smiles", "selfies"]]
drp_test_df.head()

In [ ]:
drp_test_df.to_csv(os.path.join("data", "drp_test.csv"), index=False)

In [ ]:
# save the scaler checkpoint
joblib.dump(scaler, os.path.join("data", "scaler_checkpoints", "drp_scaler.joblib"))

## Checking Nan and Inf values

### GE dataset

In [ ]:
import numpy as np

In [ ]:
ge_df = pd.read_csv(os.path.join("data", "ge_filtered_scaled.csv"), header=0, index_col=0)
ge_df.head()

In [ ]:
ge_df.isna().any().any()

In [ ]:
# test inf values
ge_df[ge_df == np.inf].any().any()

In [ ]:
# test -inf values
ge_df[ge_df == -np.inf].any().any()

In [ ]:
# test value range
ge_df.max().max(), ge_df.min().min()

Seems like they are huge outliers

In [ ]:
# compute mean and std of each column then check if there are which are not 1 and 0
ge_df.mean().mean(), ge_df.std().mean()

### DRP dataset

In [ ]:
drp_df = pd.read_csv(os.path.join("data", "drp_train.csv"), header=0)
drp_df.head()

In [ ]:
auc_values = drp_df["area_under_curve_scaled"]
auc_values.max(), auc_values.min()

In [ ]:
auc_values.mean(), auc_values.std()

In [ ]:
auc_values.isna().any()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.displot(auc_values, kde=True, height=8, aspect=2)
plt.show()